# Video Generation: Last 15 Periods

This notebook generates a 15-second video showing the last 15 periods of a specified simulation using the cuda-filaments analysis tools.

## 1. Import Required Libraries

Import necessary libraries including numpy, matplotlib, and the plotting_and_analysis_functions module.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
import matplotlib.colors as mcolors
from pathlib import Path
import sys

# Add the analysis directory to path if needed
sys.path.insert(0, '/data/pz723/cuda-filaments/analysis')

from plotting_and_analysis_functions import (
    load_simulation,
    plot_sphere_surface,
    add_phase_legend,
    DEFAULT_CMAP
)

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Set Simulation Parameters

Define the simulation file path (base_path), number of steps per period, and video output settings (fps, dpi).

In [2]:
# Simulation parameters
base_path = "../data/ablate_0.0_0/20251216/ciliate_216fil_40962blob_8.00R_0.1500torsion_0.2182tilt_0.3000f_eff_1.4960theta0_0.0000freqshift"  # UPDATE THIS PATH
num_steps = 100  # Steps per period - UPDATE THIS VALUE

# Video settings
fps = 25  # Frames per second for 15-second video
dpi = 180  # Resolution
video_duration = 15  # seconds
num_periods = 15  # Number of periods to show

# Output path
output_dir = Path("/home/pz723/cuda-filaments/analysis/analysis_output")
output_dir.mkdir(exist_ok=True, parents=True)
output_path = output_dir / "last_15_periods.mp4"

print(f"Base path: {base_path}")
print(f"Steps per period: {num_steps}")
print(f"Video duration: {video_duration}s at {fps} fps = {video_duration * fps} total frames")
print(f"Output: {output_path}")

Base path: ../data/ablate_0.0_0/20251216/ciliate_216fil_40962blob_8.00R_0.1500torsion_0.2182tilt_0.3000f_eff_1.4960theta0_0.0000freqshift
Steps per period: 100
Video duration: 15s at 25 fps = 375 total frames
Output: /home/pz723/cuda-filaments/analysis/analysis_output/last_15_periods.mp4


## 3. Load Simulation Data

Load the simulation data using the load_simulation function, which reads phase data, segment positions, and basal positions.

In [3]:
# Load simulation data
print("Loading simulation data...")
sim = load_simulation(base_path, num_steps=num_steps)

print(f"✓ Loaded simulation data:")
print(f"  - Total time steps: {sim.phases.shape[0]}")
print(f"  - Number of cilia: {sim.num_fils}")
print(f"  - Segments per cilium: {sim.num_segs}")
print(f"  - Time range: {sim.times[0]:.2f} to {sim.times[-1]:.2f} periods")
print(f"  - Sphere radius: {sim.sphere_radius:.2f}")

Loading simulation data...
[info] Using num_steps=100 for time normalization.
✓ Loaded simulation data:
  - Total time steps: 4014
  - Number of cilia: 216
  - Segments per cilium: 20
  - Time range: 0.00 to 200.65 periods
  - Sphere radius: 199.90


## 4. Calculate Frame Indices for Last 15 Periods

Determine the frame indices corresponding to the last 15 periods of the simulation based on the total time steps and steps per period.

In [4]:
# Calculate frame range for last 15 periods
total_frames = sim.phases.shape[0]
frames_per_period = num_steps

# Calculate start frame for last 15 periods
start_frame = max(0, total_frames - (num_periods * frames_per_period))
end_frame = total_frames

# Calculate stride to achieve desired video duration
# We want video_duration * fps frames in the output
total_output_frames = video_duration * fps
available_frames = end_frame - start_frame
stride = max(1, available_frames // total_output_frames)

# Actual frames that will be rendered
frame_indices = range(start_frame, end_frame, stride)
actual_output_frames = len(frame_indices)

print(f"Frame calculation:")
print(f"  - Total simulation frames: {total_frames}")
print(f"  - Frames per period: {frames_per_period}")
print(f"  - Last 15 periods span: frames {start_frame} to {end_frame} ({available_frames} frames)")
print(f"  - Stride: {stride}")
print(f"  - Output frames: {actual_output_frames} (target: {total_output_frames})")
print(f"  - Actual video duration: {actual_output_frames / fps:.2f}s")
print(f"  - Time range: {sim.times[start_frame]:.2f} to {sim.times[end_frame-1]:.2f} periods")

Frame calculation:
  - Total simulation frames: 4014
  - Frames per period: 100
  - Last 15 periods span: frames 2514 to 4014 (1500 frames)
  - Stride: 4
  - Output frames: 375 (target: 375)
  - Actual video duration: 15.00s
  - Time range: 125.70 to 200.65 periods


## 5. Generate Video with Custom Frame Range

Create a modified version of make_topdown_video that accepts a frame range parameter, then generate the 15-second video showing only the last 15 periods with appropriate stride to achieve the desired duration.

In [5]:
def make_topdown_video_custom_range(sim, 
                                    output_path,
                                    frame_indices,
                                    fps=25,
                                    cmap=DEFAULT_CMAP,
                                    dpi=180,
                                    progress=True):
    """
    Generate MP4 top-down animation colored by phase for a custom frame range.
    
    Args:
        sim: SimulationData object
        output_path: Path for output video file
        frame_indices: Iterable of frame indices to include
        fps: Frames per second
        cmap: Colormap for phase coloring
        dpi: Video resolution
        progress: Show progress bar if available
    """
    output_path = Path(output_path)
    output_path.parent.mkdir(exist_ok=True, parents=True)
    
    norm = mcolors.Normalize(vmin=0, vmax=2*np.pi)
    
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(111, projection='3d')
    
    # Setup axes and sphere
    x_all = sim.seg_positions[..., 0]
    y_all = sim.seg_positions[..., 1]
    z_all = sim.seg_positions[..., 2]
    margin = 2.0
    ax.set_xlim(np.min(x_all) - margin, np.max(x_all) + margin)
    ax.set_ylim(np.min(y_all) - margin, np.max(y_all) + margin)
    ax.set_zlim(np.min(z_all) - margin, np.max(z_all) + margin)
    ax.view_init(elev=90, azim=-90)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_zticklabels([])
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_zlabel("")
    ax.set_title("")
    
    plot_sphere_surface(ax, sim.sphere_radius, alpha=0.15)
    add_phase_legend(fig, ax, cmap=cmap, f_eff=0.3)
    
    lines = [ax.plot([], [], [], '-', lw=2)[0] for _ in range(sim.phases.shape[1])]
    
    def init():
        for ln in lines:
            ln.set_data([], [])
            ln.set_3d_properties([])
        return lines
    
    def update(f):
        XYZ = sim.seg_positions[f]
        for i, ln in enumerate(lines):
            ln.set_data(XYZ[i, :, 0], XYZ[i, :, 1])
            ln.set_3d_properties(XYZ[i, :, 2])
            ln.set_color(cmap(norm(sim.phases[f, i])))
        return lines
    
    ani = FuncAnimation(fig, update, frames=frame_indices, init_func=init, blit=False)
    writer = FFMpegWriter(fps=fps, metadata=dict(artist="cuda-filaments"))
    
    if progress:
        try:
            from tqdm import tqdm
            with tqdm(total=len(frame_indices), desc=f"Rendering {output_path.name}") as pbar:
                ani.save(output_path.as_posix(), writer=writer, dpi=dpi,
                        progress_callback=lambda i, n: pbar.update(1))
        except ImportError:
            print("tqdm not available, rendering without progress bar...")
            ani.save(output_path.as_posix(), writer=writer, dpi=dpi)
    else:
        ani.save(output_path.as_posix(), writer=writer, dpi=dpi)
    
    plt.close(fig)
    return output_path

# Generate the video
print(f"\nGenerating video...")
print(f"This may take several minutes depending on the number of frames and resolution...")

result_path = make_topdown_video_custom_range(
    sim=sim,
    output_path=output_path,
    frame_indices=frame_indices,
    fps=fps,
    cmap=DEFAULT_CMAP,
    dpi=dpi,
    progress=True
)

print(f"\n✓ Video saved to: {result_path}")
print(f"  Duration: {actual_output_frames / fps:.2f}s")
print(f"  Resolution: {dpi} DPI")
print(f"  Frame rate: {fps} fps")


Generating video...
This may take several minutes depending on the number of frames and resolution...


Rendering last_15_periods.mp4: 100%|██████████| 375/375 [02:41<00:00,  2.32it/s]


✓ Video saved to: /home/pz723/cuda-filaments/analysis/analysis_output/last_15_periods.mp4
  Duration: 15.00s
  Resolution: 180 DPI
  Frame rate: 25 fps


## Summary

The video has been generated showing the last 15 periods of the simulation. 

**Key points:**
- The video shows cilia colored by their phase (ψ) in a top-down view
- Phase coloring uses a periodic colormap where similar colors indicate similar phases
- The circular phase legend in the corner shows the phase color mapping
- Each cilium is rendered as a line from its basal position through all its segments

**To use this notebook:**
1. Update the `base_path` variable in Section 2 with your simulation data path
2. Update `num_steps` to match your simulation's steps per period
3. Run all cells to generate the video
4. The output video will be saved to `analysis_output/last_15_periods.mp4`